<h1>BigQuery ML で出生時体重を予測する</h1>

このチュートリアルでは、データ アナリストの方を対象に BigQuery ML を紹介します。BigQuery ML を使用すると、BigQuery で SQL クエリを使用して機械学習モデルを作成して実行できます。このチュートリアルは、SQL のユーザーが簡単に機械学習を利用できるようにすることを目標としています。使い慣れたツールを使用してモデルを構築でき、データを移動する必要もないため、開発スピードを向上させることができます。


このチュートリアルでは、[natality サンプル テーブル](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=samples&t=natality&page=table&hl=ja&_ga=2.232778077.336887729.1589073706-431044393.1585885350&_gac=1.219743851.1587955074.CjwKCAjw4pT1BRBUEiwAm5QuR4lYhZuwjdvkKXaQcmeIKwclqw8o3ZXPRBCThiC8R0r-EKgVvMDCJhoCbAwQAvD_BwE)を使用して、新生児の性別、妊娠期間、母親に関する人口統計情報に基づいて、出生時体重を予測するモデルを作成します。natality サンプル テーブルには、米国での新生児に関する 40 年以上の情報が格納されています。

<h2>目標</h2>
このチュートリアルでは、次のものを使用します。

- BigQuery ML。CREATE MODEL ステートメントを使用して線形回帰モデルを作成します。
- ML.EVALUATE 関数。ML モデルを評価します。
- ML.PREDICT 関数。ML モデルを使用して予測を行います。

<h2>ステップ 1: データセットを作成する</h2>

In [ ]:
%%bash
bq mk --location us bqml_tutorial

<h2>ステップ 2: モデルを作成する</h2>
次に、BigQuery の natality サンプル テーブルを使用して線形回帰モデルを作成します。子供の出生時体重の予測に使用するモデルを作成するために、次の標準 SQL クエリを使用します。

In [ ]:
%%bigquery

CREATE MODEL `bqml_tutorial.natality_model`
OPTIONS
  (model_type='linear_reg',
    input_label_cols=['weight_pounds']) AS
SELECT
  weight_pounds,
  is_male,
  gestation_weeks,
  mother_age,
  CAST(mother_race AS string) AS mother_race
FROM
  `bigquery-public-data.samples.natality`
WHERE
  weight_pounds IS NOT NULL
  AND RAND() < 0.001

`CREATE MODEL` コマンドはモデルを作成するとともに、作成したモデルのトレーニングも行います。

<h2>クエリの詳細</h2>

[CREATE MODEL](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create?hl=ja) 句は、`bqml_tutorial.natality_model` という名前のモデルを作成し、トレーニングするために使用されます。

`OPTIONS(model_type='linear_reg', input_label_cols=['weight_pounds'])` 句は、[線形回帰](https://ja.wikipedia.org/wiki/%E7%B7%9A%E5%BD%A2%E5%9B%9E%E5%B8%B0)モデルを作成することを意味します。線形回帰は、入力特徴の線型結合から連続値を生成する回帰モデルです。`weight_pounds` 列は入力ラベルの列です。線形回帰モデルの場合、ラベル列は実数にする必要があります（列の値が実数でなければなりません）。

このクエリの `SELECT` ステートメントで次の列を取得します。これらの列を使用して子供の出生時体重を予測します。

- `weight_pounds` - ポンド単位の子供の体重（FLOAT64）。
- `is_male` - 子が男児の場合は TRUE、女児の場合は FALSE（BOOL）。
- `gestation_weeks` - 妊娠週数（INT64）。
- `mother_age` - 報告された出産時の母親の年齢（INT64）。
- `mother_race` - 母親の人種に対応する整数値（INT64 - テーブル スキーマ内の `child_race` と同じ）。BigQuery ML で `mother_race` を非数値の特徴として扱うには、各カテゴリを表す固有値を使用して、クエリで `mother_race` を `STRING` 型にキャストします。人種は、（順序とスケールを持つ）整数として扱うよりも、カテゴリとして扱ったほうが良い場合が多いため、この処理は重要になります。

`FROM` 句（`bigquery-public-data.samples.natality`）は、サンプル データセットの natality サンプル テーブルにクエリを実行していることを示します。このデータセットは `bigquery-public-data` プロジェクト中にあります。

`WHERE` 句（`WHERE weight_pounds IS NOT NULL AND RAND() < 0.001`）は、体重が `NULL` の行を除外し、`RAND` 関数を使用してサンプルデータをランダムに取得します。



<h2>ステップ 3: トレーニングの統計情報を取得する</h2>

モデルのトレーニングの結果を確認するには、[`ML.TRAINING_INFO`](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-train) 関数を使用するか、BigQuery ウェブ UI で統計情報を表示します。

機械学習アルゴリズムは、多くのサンプルを検査し、損失を最小限に抑えるモデルを見つけることでモデルを構築します。

損失(`loss`)とは、精度の低い予測に対するペナルティです。これは、1 つのサンプルで予測の精度がどのくらい低いかで表します。モデルの予測が完璧であれば、損失はゼロになります。それ以外の場合、精度に応じて損失が大きくなります。モデルをトレーニングする目的は、すべてのサンプルで平均的に損失の少ない重みとバイアスの組み合わせを見つけることです。

`CREATE MODEL` クエリで生成したモデルのトレーニング統計を確認するには、次のセルを実行します。

In [ ]:
%%bigquery
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `bqml_tutorial.natality_model`)

`loss` 列は、トレーニング データセットでモデルのトレーニングを行った後に計算された損失指標を表します。線形回帰を行ったので、この列は平均二乗誤差になります。`eval_loss` 列は、ホールドアウト データセット（モデルを検証するためにトレーニングから返され、保存されているデータ）で計算された損失指標と同じです。


<h2>ステップ 4: モデルを評価する</h2>
モデルを作成したら、`ML.EVALUATE` 関数を使用してモデルの性能を評価します。`ML.EVALUATE` 関数は、実際のデータに対する予測値を評価します。

モデルの評価に使用するクエリは次のとおりです。

In [ ]:
%%bigquery
SELECT
  *
FROM
  ML.EVALUATE(MODEL `bqml_tutorial.natality_model`,
    (
    SELECT
      weight_pounds,
      is_male,
      gestation_weeks,
      mother_age,
      CAST(mother_race AS STRING) AS mother_race
    FROM
      `bigquery-public-data.samples.natality`
    WHERE
      weight_pounds IS NOT NULL))

また、入力データを指定せずに ML.EVALUATE を呼び出すこともできます。これはトレーニングで計算された評価指標を使用します。



In [ ]:
%%bigquery
SELECT
  *
FROM
  ML.EVALUATE(MODEL `bqml_tutorial.natality_model`)

線形回帰を使用しているため、結果には次の列が含まれます。

- mean_absolute_error
- mean_squared_error
- mean_squared_log_error
- median_absolute_error
- r2_score
- explained_variance

評価結果における重要な測定指標は、[R2 スコア](https://ja.wikipedia.org/wiki/%E6%B1%BA%E5%AE%9A%E4%BF%82%E6%95%B0)です。<br>R2 スコアは、線形回帰予測が実際のデータに近似するかどうかを決定する統計的尺度です。0 は、平均値周辺のレスポンス データにばらつきがないことを示しています。1 は、平均値周辺のレスポンス データにばらつきがあることを示しています。



<h2>ステップ 5: モデルを使用して結果を予測する</h2>

モデルの評価を行ったので、モデルを使用して結果を予測します。このモデルを使用して、ワイオミング州で生まれる赤ちゃんの出生時体重を予測します。

結果の予測に使用するクエリは次のとおりです。

In [ ]:
%%bigquery result
SELECT
  predicted_weight_pounds,
  struct(is_male, gestation_weeks,mother_age, mother_race) as feature
FROM
  ML.PREDICT(MODEL `bqml_tutorial.natality_model`,
    (
    SELECT
      is_male,
      gestation_weeks,
      mother_age,
      CAST(mother_race AS STRING) AS mother_race
    FROM
      `bigquery-public-data.samples.natality`
    WHERE
      state = "WY"))

In [ ]:
result.head(10)

<h2>クエリの詳細</h2>

先頭の SELECT ステートメントで、`predicted_weight_pounds` 列を取得します。<br>
この列は、`ML.PREDICT` 関数によって生成されます。`ML.PREDICT` 関数を使用した場合は、モデルの出力列名は `predicted_<label_column_name>` です。線形回帰モデルの場合、`predicted_label` は `label` の推定値になります。

`ML.PREDICT` 関数は、モデル `bqml_tutorial.natality_model` を使用して結果を予測するために使われます。

---

## ラボ: NY のタクシー料金を予測する回帰モデルを作成する

`nyc-tlc.yellow.trips` データセットを利用して、タクシー料金 (`total_fare`) を予測する回帰モデルを作成してください。<br>
その際、元のデータセットのサイズは非常に大きいので、以下のクエリを使用して、10000件に一件のデータに削減したデータをトレーニングに使用します。<br>
（Evaluation の際には、クエリの `param.TRAIN` を `param.EVAL` に変更して使用してください）

In [ ]:
%%bigquery df
WITH params AS (
   SELECT
   1 AS TRAIN,
   2 AS EVAL
   ),

 daynames AS
   (SELECT ['Sun', 'Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat'] AS daysofweek),

 taxitrips AS (
 SELECT
   (tolls_amount + fare_amount) AS total_fare,
   daysofweek[ORDINAL(EXTRACT(DAYOFWEEK FROM pickup_datetime))] AS dayofweek,
   EXTRACT(HOUR FROM pickup_datetime) AS hourofday,
   pickup_longitude AS pickuplon,
   pickup_latitude AS pickuplat,
   dropoff_longitude AS dropofflon,
   dropoff_latitude AS dropofflat,
   passenger_count AS passengers
 FROM
   `nyc-tlc.yellow.trips`, daynames, params
 WHERE
   trip_distance > 0 AND fare_amount > 0
   AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))),10000) = params.TRAIN
 )

 SELECT *
 FROM taxitrips

In [ ]:
df

### モデルを作成する

### モデルの統計情報を取得する

### モデルを評価する

Except as otherwise noted, the content of this page is licensed under the Creative Commons Attribution 4.0 License, and code samples are licensed under the Apache 2.0 License. For details, see the Google Developers Site Policies. Java is a registered trademark of Oracle and/or its affiliates.

source: [https://cloud.google.com/bigquery-ml/docs/bigqueryml-natality](https://cloud.google.com/bigquery-ml/docs/bigqueryml-natality)